## An attempt at a more formal grammar

```
function_to_call  ::= <wordcharacters>
parameters        ::= "" | "(" ( wordcharacters | number ) ")"
as_name           ::= "" | "as" <whitespace> <wordcharacters>
column_name       ::= as_name | function_to_call
reference         ::= "" |  "[" number "]"
unique_mark       ::= "" | "*"
column_definition ::= <function_to_call> <parameters> <whitespace> \
                      <as_name> <whitespace> <reference> <unique_mark>
df_size           ::= "" |  "[" integer "]"
df_sep            ::= "--" ("-"*)
df_definition     ::= <wordcharacters> <df_size> <newline> <df_sep> <newline> \
                      (<column_definition>*) <newline> <newline>
language_spec     ::= <def_definition>*
```


In [1]:
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)
import pandas as pd
from faker import Faker
import re
import random

# Config for generator
# Please tweak these numbers sensibly, keeping in mind unique faker values and reference linking
NUMROWS = 99           # number of rows per DataFrame
MAX_REPEATS = 4        # maximum number of times a reference can repeat (1 = repeated at most once)
ORPHANED_UNIQUES = 0.2 # % of uniques references which won't be used 

# Sanity check against crazy configs
assert NUMROWS < 1000, "Putting this here because there aren't enough last names for the default example"
assert (1-ORPHANED_UNIQUES) * MAX_REPEATS >= 1, "Can't generate enough data with these settings (ORPHANED_UNIQUES, MAX_REPEATS)"

# Global variables
reference_dict = {} # persistent reference for the reference columns, dict of sets
fake = Faker()

# Helper functions for casting parameters to int, float or string
def cast_parameter(x):
    if x is None:
        return x
    elif type(x) != str:
        raise ValueError('Input must be a string or None')
    try:
        a = float(x)
        b = int(a)
    except ValueError:
        pass
    else:
        if a == b:
            return b
    try:
        a = float(x)
    except ValueError:
        return x
    else:
        return a

# Pass set() to set_for_uniqueness for uniqueness requirement
# Pass None to set_for_uniqueness for no uniqueness requirement
def gen_data(_, function_name,parameter,set_for_uniqueness):
    # QUESTION: not sure if passing by set() by reference for set_for_uniqueness will mess up by panda's
    #           concurrency. It SEEMS OK, but have not tested rigorously
    # TODO:     parameter currently only takes 1 variable, need to convert to support multiple?
    # TODO:     For some function_name like first_name, we can directly get the Provider list of possible
    #           values and get a subset from it rather than repeatedly generating them
    #               from faker.providers.person.en import Provider
    #               set(Provider.first_names)
    func = getattr(fake, function_name)
    parameter = cast_parameter(parameter)
    while(True):
        value = func() if parameter is None else func(parameter)
        if type(set_for_uniqueness) is set:
            if value not in set_for_uniqueness:
                set_for_uniqueness.add(value)
                return value
        else:
            return value

def get_reference_column_data(reference_key, function_name,parameter,is_unique):
    # ASSUMPTION: assuming that function_name and parameter for each reference is configured the same way
    # first generate unique set of data if it doesn't already exist
    if reference_key not in reference_dict.keys():
        new_data = set()
        while(len(new_data)<NUMROWS):
            new_data.add(gen_data(None, function_name,parameter,new_data))
        reference_dict[reference_key] = new_data

    if is_unique:
        # if we are just looking for the unique data, we can returned the shuffled set
        return random.sample(reference_dict[reference_key], len(reference_dict[reference_key]))
    else:
        # sample items to exclude some reference values
        values = random.sample(reference_dict[reference_key], int(NUMROWS*(1-ORPHANED_UNIQUES)))
        # return sampled items (so they appear at least once) and fill the rest with the same sampled items
        # based on the MAX_REPEATS configuration
        return values + random.sample(values * (MAX_REPEATS-1),NUMROWS - len(values))

@register_cell_magic
def fakedata(line, cell):

    # do the DataFrames one by one
    pattern = r'([\w]+)(?:\n-+\n)((?:.+\n?)+)'
    match = re.findall(pattern, cell.strip())
    
    #looping through all the DataFrames
    for (df_name, df_def) in match:
        definitions = df_def.strip().split('\n')
        column_info = {}
        
        # break down each line of a DataFrame based on provided definitions
        for d in definitions:
            dpattern = r"^(?P<function>\w+)(?:(?:\((?P<parameter>[-+]?\d*\.?\d+|\w+)\))?)(?: as (?P<as_name>\w*))?(?: \[(?P<reference>\d+)\])?(?P<unique_mark>\*)?$"
            dmatch = re.search(dpattern,d).groupdict()
            column_info[dmatch['as_name'] if dmatch['as_name'] is not None else dmatch['function']] = dmatch
            
        # create DataFrame 
        df = pd.DataFrame(index=range(0,NUMROWS), columns=column_info.keys())
        
        #populate each column with data
        for k,v in column_info.items():
            if v['reference'] is not None:
                # this is a reference column, we need to reference our references of the reference
                # since we're demonstrating One to Many
                df[k] = get_reference_column_data(v['reference'],v['function'],v['parameter'],v['unique_mark'] is not None)
            else:
                df[k] = df[k].apply(gen_data, args=(v['function'],v['parameter'],None if v['unique_mark'] is None else set()))
        
        # assign the created DataFrame as a global variable using the provided name
        globals()[df_name] = df

In [2]:
%%fakedata
persons
-------
first_name
last_name*
phone_number
random_number(5) as customer_number [1]*

purchases
---------
isbn10
credit_card_full
random_number(3) as price
random_number(5) as customer_number [1]

In [3]:
persons

,first_name,last_name,phone_number,customer_number
0,David,Smith,(624)581-4008,79415
1,Alexander,Mann,114.191.0656x597,82016
2,Diane,Casey,698.637.5153x38774,70487
3,Catherine,Flores,327.072.8807,7465
4,Casey,Berry,+1-190-439-9374x151,24348
...,...,...,...,...
94,John,Wilson,+1-706-197-3222x408,9281
95,April,Gregory,186-989-4325x5090,61281
96,William,Morris,001-987-149-7454x3818,89082
97,Mark,Phelps,001-699-352-5630,60458


In [4]:
purchases

,isbn10,credit_card_full,price,customer_number
0,0-918750-22-9,JCB 15 digit\nEric Stuart\n213187174712396 07/...,382,60207
1,1-09-396771-4,VISA 16 digit\nKevin Bailey\n4466068026304697 ...,439,65007
2,1-188-04032-4,American Express\nPaul Andrade\n37205780037858...,289,70487
3,0-7482-2935-3,American Express\nLaura Vargas\n34494893206517...,714,67013
4,1-902045-94-7,VISA 13 digit\nHannah Martin\n4702312159244 01...,614,52662
...,...,...,...,...
94,1-82882-438-0,Mastercard\nBradley Smith\n2232317925589776 09...,803,17800
95,1-293-22523-1,Discover\nKatie Frederick\n6011083714071966 12...,735,84746
96,1-938750-74-8,Maestro\nDenise Norman\n639000790878 03/24\nCV...,964,27825
97,1-231-96135-X,VISA 16 digit\nJoshua Ross\n4334863835781519 0...,109,89082


In [5]:
# stats for purchases['customer_number'] reference values
# how many times distribution of number of times each customer_number appeared
purchases['customer_number'].value_counts().value_counts()

1    63
2    12
3     4
Name: customer_number, dtype: int64

In [6]:
# # testing my cast_parameter() helper function
# l = ["-1","12313","-1.0.0","1.546","-.4","+31","+.7","abc123","d7f8g8h8jh","1e2","-6e-3","", None]
# for v in l:
#     a = cast_parameter(v)
#     print("{} value: [{}]".format(type(a),a))